In [44]:
import numpy as np
import pandas as pd

prod_df = pd.read_csv('prodPrueba1.csv', engine='python')
prog_df = pd.read_csv('progTotal.csv')

prod_df = prod_df.dropna()

# Recorte de Fecha de prod
prod_df['Fecha Cita'] = prod_df['Fecha Cita'].str[0:10]

# Columna de fecha a formato fecha:
#temp_series = pd.Series()

#temp_series = prod_df['Fecha Cita']
#prod_df['Fecha Cita'] = pd.to_datetime(temp_series)

#homologacion de PRODUCCION

prod_df['temp'] = prod_df['Tratamiento'].str.contains('MENSUALIDAD ORTODONCIA METALICA')
prod_df.loc[prod_df['temp'] == True,'ttmto_homol'] = 'MENSUALIDAD ORTODONCIA METALICA'

prod_df['temp'] = prod_df['Tratamiento'].str.contains('RETIRO BRACKETS ARCADA')
prod_df.loc[prod_df['temp'] == True,'ttmto_homol'] = 'RETIRO BRACKETS ARCADA'

prod_df['temp'] = prod_df['Tratamiento'].str.contains('RETIRO BRACKETS ARCADA')
prod_df.loc[prod_df['temp'] == True,'ttmto_homol'] = 'RETIRO BRACKETS ARCADA'

prod_df['temp'] = prod_df['Tratamiento'].str.contains('PROFILAXIS')
prod_df.loc[prod_df['temp'] == True,'ttmto_homol'] = 'PROFILAXIS'

prod_df['temp'] = prod_df['Tratamiento'].str.contains('DETARTRAJE')
prod_df.loc[prod_df['temp'] == True,'ttmto_homol'] = 'DETARTRAJE POR ARCADA'

prod_df['temp'] = prod_df['Tratamiento'].str.contains('ESSIX')
prod_df.loc[prod_df['temp'] == True,'ttmto_homol'] = 'RETENEDOR ESSIX'

prod_df

,Clínica,Fecha Cita,Fecha Pago,DNI,Doctor,Paciente,Nº Historia,Presupuesto,Tratamiento,Categoría,Especialidad,Duración,Importe Tratamiento,% Descuento,Importe Descuento,Importe Producido,Equivalencia,temp,ttmto_homol
0,Autopista 104A Bogota,02/01/2016,02/01/2016,111,MARIA ISABEL MUÑOZ QUINTERO,EVELYN DAZA,123,10,1x RETIRO BRACKETS ARCADA INFERIOR. Pieza: 11,Generalista,OPERATORIA GENERAL,30,90000,0%,0,90000,0,False,RETIRO BRACKETS ARCADA


In [45]:
prod_df['Cant'] = (prod_df['Importe Producido'] + prod_df['Importe Descuento'])/prod_df['Importe Tratamiento']
prodResumen = prod_df.groupby(['Clínica','Paciente','DNI','ttmto_homol'], as_index = False).sum()
prodResumen = prodResumen.rename(index=str, columns={"Clínica": "Nombre_Clinica"})

prodResumen[['Nombre_Clinica','Paciente','DNI','ttmto_homol','Cant']]

,Nombre_Clinica,Paciente,DNI,ttmto_homol,Cant
0,Autopista 104A Bogota,EVELYN DAZA,111,RETIRO BRACKETS ARCADA,1.0


In [46]:
# CONCIDENCIAS ENTRE VENTAS Y PROD
inner_df = pd.merge(prog_df, prodResumen, how='inner', left_on=['Nombre_Clinica','Paciente','dni','ttmto_homol'], right_on=['Nombre_Clinica','Paciente','DNI','ttmto_homol'])

# ACTUALIZACION DE CANTIDADES PRODUCIDAS Y PENDTES
inner_df['Cant_prod'] = inner_df['Cant_prod'] + inner_df['Cant']
inner_df['Cant_pend'] = inner_df['Cantidad'] - inner_df['Cant_prod']
#inner_df[['Nombre_Clinica','Paciente','dni','ttmto_homol','Cantidad','Cant_prod','Cant_pend']]
inner_df


,Nombre_Clinica,Paciente,dni,ttmto_homol,Cantidad,Cant_prod,Cant_pend,fechaVenta,Semana_prog,fechaProg,DNI,Nº Historia,Presupuesto,Duración,Importe Tratamiento,Importe Descuento,Importe Producido,Equivalencia,temp,Cant
0,Autopista 104A Bogota,EVELYN DAZA,111,RETIRO BRACKETS ARCADA,2,1.0,1.0,01/02/2018,1,01/02/2018,111,123,10,30,90000,0,90000,0,False,1.0
